In [1]:
# Install required libraries
!pip install ultralytics

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 22.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 42.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 31.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 42.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 13.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 45.0 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling 

In [2]:
import cv2
import numpy as np
from ultralytics import YOLO
from google.colab import drive


Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.


In [4]:
# Mount Google Drive
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
# Load YOLOv8 model
model = YOLO('yolov8n.pt')

# Load the video from Google Drive
cap = cv2.VideoCapture('/content/drive/MyDrive/Traking_video/input_video1.mp4')
if not cap.isOpened():
    print("Error: Could not open video file")
    exit()

# Get video properties
frame_width = int(cap.get(3))
frame_height = int(cap.get(4))
fps = int(cap.get(5))
print(f"Video properties - Width: {frame_width}, Height: {frame_height}, FPS: {fps}")

# Define the codec and create VideoWriter object
fourcc = cv2.VideoWriter_fourcc(*'XVID')
out = cv2.VideoWriter('/content/drive/MyDrive/Traking_video/output_video.mp4', fourcc, fps, (frame_width, frame_height))
if not out.isOpened():
    print("Error: Could not open output video writer")
    exit()

# Define colors for tracked persons
colors = [
    (0, 0, 255), (0, 255, 0), (255, 0, 0), (255, 255, 0), (0, 255, 255),
    (255, 0, 255), (128, 0, 0), (0, 128, 0), (0, 0, 128), (128, 128, 0),
    (128, 0, 128), (0, 128, 128), (255, 128, 0), (255, 0, 128), (128, 255, 0),
    (0, 255, 128), (128, 0, 255), (0, 128, 255), (255, 128, 128), (128, 255, 128)
]

# Dictionary to store track history for each person
track_history = {}

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        print("Failed to read frame")
        break
    print("Processing frame")

    # Run tracking on the current frame with ByteTrack
    results = model.track(frame, persist=True, classes=[0], tracker='bytetrack.yaml')

    for result in results:
        boxes = result.boxes.xyxy.cpu().numpy()
        ids = result.boxes.id
        if ids is None:
            continue
        ids = ids.cpu().numpy()

        for box, id in zip(boxes, ids):
            id = int(id)
            x1, y1, x2, y2 = map(int, box)
            center = ((x1 + x2) // 2, (y2 + y1) // 2)

            # Initialize track history if new ID
            if id not in track_history:
                color_index = len(track_history) % len(colors)
                track_history[id] = {'points': [center], 'color': colors[color_index]}
            else:
                track_history[id]['points'].append(center)

            # Draw bounding box
            cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 2)
            cv2.putText(frame, f'Person {id}', (x1, y1-10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)

            # Draw path
            if len(track_history[id]['points']) > 1:
                color = track_history[id]['color']
                for i in range(1, len(track_history[id]['points'])):
                    if track_history[id]['points'][i-1] and track_history[id]['points'][i]:
                        cv2.line(frame, track_history[id]['points'][i-1], track_history[id]['points'][i], color, 2)

    # Write the frame to output video
    out.write(frame)

# Release resources
cap.release()
out.release()

print("Video processing complete. Check '/content/drive/MyDrive/Traking_video/output_video.mp4'.")

100%|██████████| 6.25M/6.25M [00:00<00:00, 136MB/s]


Streaming output truncated to the last 5000 lines.
0: 384x640 2 persons, 98.7ms
Speed: 1.5ms preprocess, 98.7ms inference, 1.3ms postprocess per image at shape (1, 3, 384, 640)
Processing frame

0: 384x640 2 persons, 97.4ms
Speed: 1.8ms preprocess, 97.4ms inference, 0.8ms postprocess per image at shape (1, 3, 384, 640)
Processing frame

0: 384x640 2 persons, 100.1ms
Speed: 1.5ms preprocess, 100.1ms inference, 0.9ms postprocess per image at shape (1, 3, 384, 640)
Processing frame

0: 384x640 2 persons, 96.1ms
Speed: 3.7ms preprocess, 96.1ms inference, 0.8ms postprocess per image at shape (1, 3, 384, 640)
Processing frame

0: 384x640 2 persons, 94.8ms
Speed: 1.6ms preprocess, 94.8ms inference, 0.8ms postprocess per image at shape (1, 3, 384, 640)
Processing frame

0: 384x640 2 persons, 93.4ms
Speed: 1.7ms preprocess, 93.4ms inference, 0.9ms postprocess per image at shape (1, 3, 384, 640)
Processing frame

0: 384x640 2 persons, 99.4ms
Speed: 1.6ms preprocess, 99.4ms inference, 0.8ms postp